# Prerequisites

In [ ]:
import numpy as np
import pandas as pd
from tm import StockDataProvider
from tm.trading_rules import SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC, BollingerBaender
from tm.optimizers import GeneticOptimizer, StrategyPerformanceEvaluator, map_chromosome_to_trading_rule_parameters, filter_for_active_rules, calculate_absolute_buy_and_hold_returns
from tm.backtesting import GeometricBrownianMotion, MonteCarloCrossValidation
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Stock Price Simulation with Geometric Brownian Motion

In [ ]:
# Use the 100 days training period of Burbach et al.
data = StockDataProvider('SAP', start='2014-11-01', end='2015-03-31')

In [ ]:
gbm = GeometricBrownianMotion(data)
simulations_df = gbm.simulate(num_simulations=100, time_steps=84)

In [ ]:
real_stock_prices = StockDataProvider('SAP', start='2015-04-01', end='2015-07-30').history['Close']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 10))
data.history['Close'].plot(ax=ax, label='Daily stock prices used for simulation', color='blue')
simulations_df.plot(ax=ax, style=':', legend=False)
real_stock_prices.plot(ax=ax, label='Real stock prices', lw=5, color='blue')

ax.set_xlabel('Date')
ax.set_ylabel('Stock price')
ax.set_title('SAP stock price Monte Carlo Simulation using Geometric Brownian Motion stochastic process');

# Optimization and Backtesting

In [ ]:
trading_rules = [SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC, BollingerBaender]

Run the optimization with the Genetic Algorithm

In [ ]:
optimizer = GeneticOptimizer(data, trading_rules, random_seed=42)

In [ ]:
%%time
hof = optimizer.run(pop_size=1000, ngen=3, hof_size=100)
print('Net profit of buy and hold:', calculate_absolute_buy_and_hold_returns(data))

Use 100-fold crossvalidation on artificially generated stock prices with geometric brownian motion

In [ ]:
crossvalidator = MonteCarloCrossValidation(hof, gbm, trading_rules)

In [ ]:
%%time
# Best individual is the bitvector representing the best strategy after crossvalidation
best_individual = crossvalidator.run(num_iterations=100, time_steps=84)
print(hof[0])

Apply the best found strategy on the real data and compare the result with buy and hold

In [ ]:
real_data = StockDataProvider('SAP', start='2015-04-01', end='2015-07-30')
rule_instances = list(map(lambda Rule, params: Rule(real_data, *params), trading_rules, map_chromosome_to_trading_rule_parameters(best_individual, trading_rules)))
active_rule_instances = filter_for_active_rules(best_individual, rule_instances)
evaluator = StrategyPerformanceEvaluator(active_rule_instances)
net_profit, last_sell = evaluator.calculate_net_profit()
print('Active rules:', list(map(lambda rule: rule.__class__.__name__, active_rule_instances)))
print('Active rules parameters:', list(map(lambda param: param[1], filter(lambda param: trading_rules[param[0]] in map(lambda rule: rule.__class__, active_rule_instances), enumerate(map_chromosome_to_trading_rule_parameters(best_individual, trading_rules))))))
print('Net profit of strategy: {:f}'.format(net_profit))
print('Net profit of buy and hold: {:f}'.format(calculate_absolute_buy_and_hold_returns(real_data, early_out=last_sell)))
print('Percentage profit of strategy: {:f}%'.format(net_profit / real_data.history['Close'].iloc[0] * 100))
print('Percentage profit of buy and hold: {:f}%'.format(calculate_absolute_buy_and_hold_returns(real_data, early_out=last_sell) / real_data.history['Close'].iloc[0] * 100))
print('Number of buy signals:', len(evaluator.buy_signals[evaluator.buy_signals == True]))
print('Number of sell signals:', len(evaluator.sell_signals[evaluator.sell_signals == True]))

Print sell signals of Bollinger Bänder stop loss.

In [ ]:
bb = rule_instances[-1]
bb_sell_signals = bb.sell_signals()
bb_sell_signals[bb_sell_signals == True]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(real_data.history['Close'], label='Stock prices')

buy_points = pd.Series(data=map(lambda index, price: price if evaluator.buy_signals.loc[index] == True else np.nan, real_data.history['Close'].index, real_data.history['Close']),
                       index=real_data.history['Close'].index)
ax.scatter(x=buy_points.index, y=buy_points, color='green', marker='^', label='Buy signal')
sell_points = pd.Series(data=map(lambda index, price: price if evaluator.sell_signals.loc[index] == True else np.nan, real_data.history['Close'].index, real_data.history['Close']),
                       index=real_data.history['Close'].index)
ax.scatter(x=sell_points.index, y=sell_points, color='red', marker='v', label='Sell signal')

ax.set_xlabel('Date')
ax.set_ylabel('Stock price')
ax.legend()
fig.tight_layout()